# <center> `Rodrigo Arroyo & Gabriel Chouraqui`
## <center> Business Challenge #1 - MBAN DD

# Importing the warnings and the libraries

In [29]:
import warnings
warnings.filterwarnings("ignore")

In [30]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Importing the data

In [31]:
data_path ="./data/"

california_filename = "current_california_state_rates.csv"

california_df_data = pd.read_csv(data_path + california_filename)

california_df = california_df_data.copy()

In [32]:
data_path ="./data/"

inflation_filename = "inflation_rate.csv"

inflation_df_data = pd.read_csv(data_path + inflation_filename)

inflation_df = inflation_df_data.copy()

In [33]:
data_path ="./data/"

federal_filename = "current_federal_tax_rates.csv"

federal_df_data = pd.read_csv(data_path + federal_filename)

federal_df = federal_df_data.copy()

In [34]:
data_path ="./data/"

mass_filename = "mass_tax.csv"

mass_df_data = pd.read_csv(data_path + mass_filename)

mass_df = mass_df_data.copy()

# Check if they need to move or to stay

In [35]:
# Function to calculate taxes based on income and tax brackets
def calculate_tax_on_income(total_income, tax_brackets):
    tax = 0
    prev_limit = 0
    for limit, rate in tax_brackets.items():
        if total_income > limit:
            tax += (limit - prev_limit) * rate
        else:
            tax += (total_income - prev_limit) * rate
            break
        prev_limit = limit
    return tax

print("Welcome to your Retirement Planner")

data_path = "./data/"

california_filename = "current_california_state_rates.csv"
federal_filename = "current_federal_tax_rates.csv"
mass_filename = "mass_tax.csv"

mass_df_data = pd.read_csv(data_path + mass_filename)

# Filter for Massachusetts tax rate
massachusetts_tax_rate = mass_df_data.loc[mass_df_data['state'] == 'Massachussetts', 'tax'].values[0]

# Calculate taxes for Massachusetts taxes on the interest earned
massachusetts_tax_brackets = {float('inf'): massachusetts_tax_rate} 

# User input for savings, spending, and investment rate
savings = float(input("Enter your savings amount: $"))
annual_spending = float(input("Enter your annual spending: $"))
investment_rate = float(input("Enter your investment rate (as a decimal): "))

# Assuming you've calculated interest_earned
interest_earned = savings * investment_rate

# Extract tax brackets for California and Federal taxes
california_df_data = pd.read_csv(data_path + california_filename)
california_tax_brackets = california_df_data.set_index('Up_to')['percentage_on_excess'].to_dict()

federal_df_data = pd.read_csv(data_path + federal_filename)
federal_tax_brackets = dict(zip(federal_df_data['Up_to'], federal_df_data['percentage_on_excess']))

# Calculate taxes for California and Federal taxes on the interest earned
california_tax_on_interest = calculate_tax_on_income(interest_earned, california_tax_brackets)
federal_tax_on_interest = calculate_tax_on_income(interest_earned, federal_tax_brackets)

# Calculate taxes for Massachusetts and Federal taxes on the interest earned
massachusetts_tax_on_interest = calculate_tax_on_income(interest_earned, massachusetts_tax_brackets)
total_massachusetts_tax_on_interest = massachusetts_tax_on_interest + federal_tax_on_interest


# Determine which location has lower taxes on interest earned (California + Federal vs Massachusetts + Federal)
if (california_tax_on_interest + federal_tax_on_interest) < total_massachusetts_tax_on_interest:
    print("Considering taxes on interest earned, it might be more beneficial to live in California.")
    tax_savings_on_interest = total_massachusetts_tax_on_interest - (california_tax_on_interest + federal_tax_on_interest)
    print(f"You might save approximately ${tax_savings_on_interest:.2f} in taxes on interest by living in California.")
else:
    print("Considering taxes on interest earned, it might be more beneficial to live in Massachusetts.")
    tax_savings_on_interest = (california_tax_on_interest + federal_tax_on_interest) - total_massachusetts_tax_on_interest
    print(f"You might save approximately ${tax_savings_on_interest:.2f} in taxes on interest by living in Massachusetts.")

Welcome to your Retirement Planner
Enter your savings amount: $580000
Enter your annual spending: $81500
Enter your investment rate (as a decimal): 0.06
Considering taxes on interest earned, it might be more beneficial to live in California.
You might save approximately $1245.98 in taxes on interest by living in California.


# Check budget by taking inflation and year

In [36]:
data_path = "./data/"
inflation_filename = "inflation_rate.csv"

def adjust_for_inflation(spending_budget, year, inflation_df):
    inflation_rates = inflation_df.loc[inflation_df['Year'] <= year, 'Inflation Rate'].values
    
    if len(inflation_rates) == 0:
        print("Inflation rate data not available for the given year.")
        return spending_budget
    
    adjusted_budget = spending_budget
    for inflation_rate in inflation_rates:
        adjusted_budget *= (1 + inflation_rate)
    
    return adjusted_budget

def get_user_input():
    while True:
        try:
            spending_budget = float(input("Enter your current annual spending budget: $"))
            if spending_budget < 0:
                raise ValueError("Spending budget cannot be negative.")
            break
        except ValueError as e:
            print(e)
    
    while True:
        try:
            year = int(input("Enter the year: "))
            break
        except ValueError:
            print("Please enter a valid year.")
    
    return spending_budget, year

def main():
    inflation_df_data = pd.read_csv(data_path + inflation_filename)
    inflation_df = inflation_df_data.copy()
    
    spending_budget, year = get_user_input()
    
    adjusted_budget = adjust_for_inflation(spending_budget, year, inflation_df)
    
    print(f"Adjusted budget considering inflation for {year}: ${adjusted_budget:.2f}")
    
if __name__ == "__main__":
    main()

Enter your current annual spending budget: $81500
Enter the year: 2030
Adjusted budget considering inflation for 2030: $105751.26


# Check for how long the savings will last considering investment rate and inflation

In [37]:
def calculate_tax_on_income(total_income, tax_brackets):
    tax = 0
    prev_limit = 0
    for limit, rate in tax_brackets.items():
        if total_income > limit:
            tax += (limit - prev_limit) * rate
        else:
            tax += (total_income - prev_limit) * rate
            break
        prev_limit = limit
    return tax

def get_state_tax_brackets(state):
    if state == 'CA':
        california_df_data = pd.read_csv("./data/current_california_state_rates.csv")
        state_tax_brackets = california_df_data.set_index('Up_to')['percentage_on_excess'].to_dict()
    elif state == 'MA':
        mass_df_data = pd.read_csv("./data/mass_tax.csv")
        massachusetts_tax_rate = mass_df_data.loc[mass_df_data['state'] == 'Massachussetts', 'tax'].values
        if len(massachusetts_tax_rate) > 0:
            massachusetts_tax_brackets = {float('inf'): massachusetts_tax_rate[0]}
        else:
            raise ValueError("Tax rate data not found for Massachusetts.")
        return massachusetts_tax_brackets
    else:
        raise ValueError("Invalid state. Please enter 'CA' for California or 'MA' for Massachusetts.")
    
    return state_tax_brackets


def calculate_years_until_depletion(initial_savings, annual_spending, inflation_rates, investment_rate, federal_tax_brackets, state_tax_brackets):
    years = 0
    savings = initial_savings
    current_year = min(inflation_rates.keys())

    while savings > 0:
        # Fetch inflation rate for the current year
        inflation_rate = inflation_rates.get(current_year, 0)

        # Adjust spending for inflation
        annual_spending *= (1 + inflation_rate)
        
        # Calculate investment returns
        investment_returns = savings * investment_rate
        
        # Calculate federal tax on investment returns
        federal_tax_on_interest = calculate_tax_on_income(investment_returns, federal_tax_brackets)
        
        # Calculate state tax on investment returns
        state_tax_on_interest = calculate_tax_on_income(investment_returns, state_tax_brackets)
        
        # Update savings considering spending, investment returns, and taxes
        savings += investment_returns - annual_spending - federal_tax_on_interest - state_tax_on_interest
        
        # Increment the number of years
        years += 1
        
        # Move to the next year
        current_year += 1
        
        # Break if the savings are depleted or inflation data is not available beyond the current year
        if savings <= 0 or current_year not in inflation_rates:
            break
    
    return years

def get_user_input(inflation_rates):
    initial_savings = float(input("Enter your initial savings amount: $"))
    annual_spending = float(input("Enter your annual spending: $"))
    investment_rate = float(input("Enter the investment rate (as a decimal): "))
    
    federal_df_data = pd.read_csv("./data/current_federal_tax_rates.csv")
    federal_tax_brackets = dict(zip(federal_df_data['Up_to'], federal_df_data['percentage_on_excess']))
    
    state = input("Enter your state (CA or MA): ")
    state_tax_brackets = get_state_tax_brackets(state)
    
    return initial_savings, annual_spending, inflation_rates, investment_rate, federal_tax_brackets, state_tax_brackets

def main():
    # Read inflation rates from the provided file
    inflation_df = pd.read_csv("./data/inflation_rate.csv")
    inflation_rates = dict(zip(inflation_df['Year'], inflation_df['Inflation Rate']))
    
    # Get user input
    initial_savings, annual_spending, inflation_rates, investment_rate, federal_tax_brackets, state_tax_brackets = get_user_input(inflation_rates)
    
    # Calculate the number of years until savings depletion
    years_until_depletion = calculate_years_until_depletion(initial_savings, annual_spending, inflation_rates, investment_rate, federal_tax_brackets, state_tax_brackets)
    
    # Print the result
    print(f"The savings will last for approximately {years_until_depletion} years.")
    
if __name__ == "__main__":
    main()


Enter your initial savings amount: $580000
Enter your annual spending: $81500
Enter the investment rate (as a decimal): 0.06
Enter your state (CA or MA): CA
The savings will last for approximately 8 years.


# Calculate the cut expenses

In [38]:
def calculate_tax_on_income(total_income, tax_brackets):
    tax = 0
    prev_limit = 0
    for limit, rate in tax_brackets.items():
        if total_income > limit:
            tax += (limit - prev_limit) * rate
        else:
            tax += (total_income - prev_limit) * rate
            break
        prev_limit = limit
    return tax

def get_state_tax_brackets(state):
    if state == 'CA':
        california_df_data = pd.read_csv("./data/current_california_state_rates.csv")
        state_tax_brackets = california_df_data.set_index('Up_to')['percentage_on_excess'].to_dict()
    elif state == 'MA':
        mass_df_data = pd.read_csv("./data/mass_tax.csv")
        massachusetts_tax_rate = mass_df_data.loc[mass_df_data['state'] == 'Massachussetts', 'tax'].values
        if len(massachusetts_tax_rate) > 0:
            state_tax_brackets = {float('inf'): massachusetts_tax_rate[0]}
        else:
            raise ValueError("Tax rate data not found for Massachusetts.")
    else:
        raise ValueError("Invalid state. Please enter 'CA' for California or 'MA' for Massachusetts.")
    
    return state_tax_brackets

def calculate_years_until_depletion(initial_savings, annual_spending, inflation_rates, investment_rate, ira_age, federal_tax_brackets, state_tax_brackets):
    years = 0
    savings = initial_savings
    current_year = min(inflation_rates.keys())
    
    years_until_ira = ira_age - current_year

    while savings > 0 and years_until_ira > 0:
        inflation_rate = inflation_rates.get(current_year, 0)
        annual_spending *= (1 + inflation_rate)
        
        investment_returns = savings * investment_rate
        
        federal_tax_on_interest = calculate_tax_on_income(investment_returns, federal_tax_brackets)
        state_tax_on_interest = calculate_tax_on_income(investment_returns, state_tax_brackets) 
        
        savings += investment_returns - annual_spending - federal_tax_on_interest - state_tax_on_interest
        
        years += 1
        current_year += 1
        years_until_ira -= 1
        
        if savings <= 0 or current_year not in inflation_rates:
            break
    
    if savings > 0 and years_until_ira <= 0:
        while savings > 0:
            inflation_rate = inflation_rates.get(current_year, 0)
            annual_spending *= (1 + inflation_rate)
            investment_returns = savings * investment_rate
            savings += investment_returns - annual_spending
            years += 1
            current_year += 1

    return years, annual_spending


def main():
    inflation_df = pd.read_csv("./data/inflation_rate.csv")
    federal_df = pd.read_csv("./data/current_federal_tax_rates.csv")
    
    initial_savings = float(input("Enter your initial savings amount: $"))
    annual_spending = float(input("Enter your annual spending: $"))
    investment_rate = float(input("Enter the investment rate (as a decimal): "))
    ira_age = 73
    state = input("Enter your state (CA or MA): ")
    
    inflation_rates = dict(zip(inflation_df['Year'], inflation_df['Inflation Rate']))
    federal_tax_brackets = dict(zip(federal_df['Up_to'], federal_df['percentage_on_excess']))
    state_tax_brackets = get_state_tax_brackets(state)
    
    years_until_depletion, reduced_annual_spending = calculate_years_until_depletion(initial_savings, annual_spending, inflation_rates, investment_rate, ira_age, federal_tax_brackets, state_tax_brackets)
    
    years_to_cut_expenses = max(0, ira_age - 62)
    amount_to_cut_per_year = (annual_spending - reduced_annual_spending) / years_to_cut_expenses if years_to_cut_expenses > 0 else 0
    
    print(f"The approximate amount to cut expenses per year: ${amount_to_cut_per_year:.2f}")
    
if __name__ == "__main__":
    main()

Enter your initial savings amount: $580000
Enter your annual spending: $81500
Enter the investment rate (as a decimal): 0.06
Enter your state (CA or MA): CA
The approximate amount to cut expenses per year: $-2204.66


# Check if the savings are sufficient to cover expenses until the life expectancy

In [39]:
def calculate_tax_on_income(total_income, tax_brackets):
    tax = 0
    prev_limit = 0
    for limit, rate in tax_brackets.items():
        if total_income > limit:
            tax += (limit - prev_limit) * rate
        else:
            tax += (total_income - prev_limit) * rate
            break
        prev_limit = limit
    return tax

def calculate_years_until_depletion(initial_savings, annual_spending, inflation_rates, investment_rate, ira_age, federal_tax_brackets, life_expectancy):
    years = 0
    savings = initial_savings
    current_year = min(inflation_rates.keys())
    
    years_until_ira = ira_age - current_year

    while savings > 0 and years_until_ira > 0 and years < life_expectancy:
        inflation_rate = inflation_rates.get(current_year, 0)
        annual_spending *= (1 + inflation_rate)
        
        investment_returns = savings * investment_rate
        
        federal_tax_on_interest = calculate_tax_on_income(investment_returns, federal_tax_brackets)
        
        savings += investment_returns - annual_spending - federal_tax_on_interest
        
        years += 1
        current_year += 1
        years_until_ira -= 1
        
        if savings <= 0 or current_year not in inflation_rates:
            break
    
    if savings > 0 and (years_until_ira <= 0 or years >= life_expectancy):
        while savings > 0 and years < life_expectancy:
            inflation_rate = inflation_rates.get(current_year, 0)
            annual_spending *= (1 + inflation_rate)
            investment_returns = savings * investment_rate
            savings += investment_returns - annual_spending
            years += 1
            current_year += 1

    return years, savings

def main():
    inflation_df = pd.read_csv("./data/inflation_rate.csv")
    federal_df = pd.read_csv("./data/current_federal_tax_rates.csv")
    
    inflation_rates = dict(zip(inflation_df['Year'], inflation_df['Inflation Rate']))
    federal_tax_brackets = dict(zip(federal_df['Up_to'], federal_df['percentage_on_excess']))


    initial_savings = float(input("Enter your initial savings amount: $"))
    annual_spending = float(input("Enter your annual spending: $"))
    investment_rate = float(input("Enter the investment rate (as a decimal): "))
    ira_age = 73  
    life_expectancy = float(input("Enter your life expectancy: ")) 
    years_until_depletion, total_savings = calculate_years_until_depletion(initial_savings, annual_spending, inflation_rates, investment_rate, ira_age, federal_tax_brackets, life_expectancy)

    if total_savings < 0:
        shortfall = abs(total_savings)
        print(f"You need to adjust your spending by approximately ${shortfall:.2f} per year to cover expenses until your life expectancy.")
    else:
        print("Your savings are sufficient to cover expenses until your life expectancy.")

if __name__ == "__main__":
    main()


Enter your initial savings amount: $580000
Enter your annual spending: $81500
Enter the investment rate (as a decimal): 0.06
Enter your life expectancy: 81
You need to adjust your spending by approximately $36126.04 per year to cover expenses until your life expectancy.


# Checking if the savings and the IRA will cover the life expectancy

In [40]:
# Function to calculate tax on income based on tax brackets
def calculate_tax_on_income(total_income, tax_brackets):
    tax = 0
    prev_limit = 0
    for limit, rate in tax_brackets.items():
        if total_income > limit:
            tax += (limit - prev_limit) * rate
        else:
            tax += (total_income - prev_limit) * rate
            break
        prev_limit = limit
    return tax

def get_state_tax_brackets(state):
    if state == 'CA':
        california_df_data = pd.read_csv("./data/current_california_state_rates.csv")
        state_tax_brackets = california_df_data.set_index('Up_to')['percentage_on_excess'].to_dict()
    elif state == 'MA':
        mass_df_data = pd.read_csv("./data/mass_tax.csv")
        massachusetts_tax_rate = mass_df_data.loc[mass_df_data['state'] == 'Massachussetts', 'tax'].values
        if len(massachusetts_tax_rate) > 0:
            state_tax_brackets = {float('inf'): massachusetts_tax_rate[0]}
        else:
            raise ValueError("Tax rate data not found for Massachusetts.")
    else:
        raise ValueError("Invalid state. Please enter 'CA' for California or 'MA' for Massachusetts.")
    
    return state_tax_brackets

# Function to calculate remaining funds based on financial data and taxes
def calculate_remaining_funds(initial_savings, ira_balance, annual_spending, ira_interest_rate, inflation_rates, federal_tax_brackets, massachusetts_tax_rate):
    remaining_funds = []
    years = 0
    age = 62

    while years <= 81.5 - 62:
        # Adjust IRA balance for interest
        ira_balance *= (1 + ira_interest_rate)
        
        if age >= 73:
            rmd = ira_balance / (81.5 - age)
            if initial_savings < annual_spending:
                ira_withdrawal = annual_spending - initial_savings
                if ira_balance >= ira_withdrawal:
                    ira_balance -= ira_withdrawal
                    initial_savings = 0
                else:
                    initial_savings -= (ira_withdrawal - ira_balance)
                    ira_balance = 0
                total_expenses = annual_spending + rmd
            else:
                rmd = 0
                total_expenses = annual_spending
        else:
            total_expenses = annual_spending
            
        # Adjust spending for inflation (considering only savings account)
        initial_savings *= (1 + inflation_rates['Inflation Rate'].iloc[years])
        
        # Apply Massachusetts tax on the 0.06 increase
        massachusetts_tax_on_interest = 0.06 * massachusetts_tax_rate
        initial_savings -= massachusetts_tax_on_interest
        
        # Apply federal tax on the 0.06 increase
        federal_tax_on_interest = calculate_tax_on_income(0.06, federal_tax_brackets)
        initial_savings -= federal_tax_on_interest
        
        # Calculate remaining funds after expenses
        remaining_funds.append(initial_savings + ira_balance - total_expenses)
        
        # Increment age and years
        age += 1
        years += 1

    return remaining_funds

# Load inflation data from CSV
inflation_df = pd.read_csv('./data/inflation_rate.csv')


# Load federal tax brackets from CSV
federal_df = pd.read_csv('./data/current_federal_tax_rates.csv')
federal_tax_brackets = dict(zip(federal_df['Up_to'], federal_df['percentage_on_excess']))

# Load Massachusetts tax rate from CSV
mass_df_data = pd.read_csv(data_path + mass_filename)
massachusetts_tax_rate = mass_df_data.loc[mass_df_data['state'] == 'Massachussetts', 'tax'].values[0]

# User input for financial variables
initial_savings = float(input("Enter your initial savings amount: $"))
ira_balance = float(input("Enter your IRA balance at 62 years old: $"))
annual_spending = float(input("Enter your annual spending: $"))
ira_interest_rate = float(input("Enter your IRA interest rate (as a decimal): "))

# Calculate remaining funds
remaining_funds = calculate_remaining_funds(initial_savings, ira_balance, annual_spending, ira_interest_rate, inflation_df, federal_tax_brackets, massachusetts_tax_rate)

# Display the results
for year, funds in enumerate(remaining_funds, start=1):
    print(f"Year {year}: Remaining funds - ${funds:.2f}")


Enter your initial savings amount: $580000
Enter your IRA balance at 62 years old: $400000
Enter your annual spending: $81500
Enter your IRA interest rate (as a decimal): 0.08
Year 1: Remaining funds - $977450.99
Year 2: Remaining funds - $1042041.93
Year 3: Remaining funds - $1099161.59
Year 4: Remaining funds - $1154205.80
Year 5: Remaining funds - $1211938.14
Year 6: Remaining funds - $1273508.31
Year 7: Remaining funds - $1340025.92
Year 8: Remaining funds - $1411457.65
Year 9: Remaining funds - $1487763.57
Year 10: Remaining funds - $1569226.12
Year 11: Remaining funds - $1656235.26
Year 12: Remaining funds - $1749227.66
Year 13: Remaining funds - $1848748.67
Year 14: Remaining funds - $1955151.80
Year 15: Remaining funds - $2068962.80
Year 16: Remaining funds - $2190748.89
Year 17: Remaining funds - $2321122.10
Year 18: Remaining funds - $2460742.81
Year 19: Remaining funds - $2610323.63
Year 20: Remaining funds - $2770633.55


# Amount of taxes taken from IRAs

In [41]:
def calculate_tax_on_income(total_income, tax_brackets):
    tax = 0
    prev_limit = 0
    for limit, rate in tax_brackets.items():
        if total_income > limit:
            tax += (limit - prev_limit) * rate
        else:
            tax += (total_income - prev_limit) * rate
            break
        prev_limit = limit
    return tax

def get_state_tax_brackets(state):
    if state == 'CA':
        california_df_data = pd.read_csv("./data/current_california_state_rates.csv")
        state_tax_brackets = california_df_data.set_index('Up_to')['percentage_on_excess'].to_dict()
    elif state == 'MA':
        mass_df_data = pd.read_csv("./data/mass_tax.csv")
        massachussetts_tax_rate = mass_df_data.loc[mass_df_data['state'] == 'Massachussetts', 'tax'].values
        if len(massachussetts_tax_rate) > 0:
            state_tax_brackets = {float('inf'): massachussetts_tax_rate[0]}
        else:
            raise ValueError("Tax rate data not found for Massachussetts.")
    else:
        raise ValueError("Invalid state. Please enter 'CA' for California or 'MA' for Massachussetts.")
    
    return state_tax_brackets

def calculate_ira_details(initial_ira_balance, ira_interest_rate, federal_tax_brackets, state_tax_brackets):
    ira_balance = initial_ira_balance
    total_withdrawn = 0
    total_taxes = 0
    age = 62

    while ira_balance > 0:
        ira_balance *= (1 + ira_interest_rate)
        rmd = ira_balance / (81.5 - age)  

        if rmd > ira_balance:
            rmd = ira_balance 
            
        federal_tax_on_rmd = calculate_tax_on_income(rmd, federal_tax_brackets)
        state_tax_on_rmd = calculate_tax_on_income(rmd, state_tax_brackets)
        total_taxes += (federal_tax_on_rmd + state_tax_on_rmd)

        total_withdrawn += rmd
        ira_balance -= (rmd + federal_tax_on_rmd + state_tax_on_rmd)
        age += 1

    return total_withdrawn, total_taxes

def main():
    # Load federal tax brackets from CSV
    federal_df = pd.read_csv('./data/current_federal_tax_rates.csv')
    federal_tax_brackets = dict(zip(federal_df['Up_to'], federal_df['percentage_on_excess']))

    # User input for state and IRA details
    state = input("Enter state (CA or MA): ")
    if state not in ['CA', 'MA']:
        print("Invalid state. Please enter 'CA' for California or 'MA' for Massachussetts.")
        return

    initial_ira_balance = float(input("Enter your IRA balance at 62 years old: $"))
    ira_interest_rate = float(input("Enter your IRA interest rate (as a decimal): "))

    # Get state tax brackets
    try:
        state_tax_brackets = get_state_tax_brackets(state)
    except ValueError as e:
        print(e)
        return

    # Calculate total withdrawn from IRA and total taxes collected
    total_withdrawn, total_taxes = calculate_ira_details(initial_ira_balance, ira_interest_rate, federal_tax_brackets, state_tax_brackets)

    # Display total withdrawn and taxes applied
    print(f"Total taxes collected from IRA proceeds until exhaustion (Federal and State): ${total_taxes:.2f}")

if __name__ == "__main__":
    main()

Enter state (CA or MA): CA
Enter your IRA balance at 62 years old: $400000
Enter your IRA interest rate (as a decimal): 0.08
Total taxes collected from IRA proceeds until exhaustion (Federal and State): $103146.78
